<h1>Sentimentanalyse der Twitter Daten</h1>

<h2>Aufgaben</h2>
<p>Laden der Daten - DONE -</p>
<p>Wie kann ich VADER nutzen um eine Sentimentanalyse durchzuführen</p>
<p>Zusammenführen der Daten</p>
<p>Aussortieren nicht benötigter Tweets und Daten</p>


In [2]:
import csv
import numpy as np
import pandas as pd
import re
import nltk
#If not downloaded:
#nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [1]:
import flair

<h2>Sentiment Test</h2>
<p>Der versuche eine einfache Sentimentanalyse mit VADER anhand von Beispieldaten durchzuführen</p>

In [2]:
sid = SentimentIntensityAnalyzer()

In [7]:
# Define a list of product reviews

product_reviews = [
    'I love this product. It helps me get so much work done. I tell everyone about what a great thing it is.',
    'This product is defective. I feel like it is broken because it does not do what it promises. Do not buy this.',
    'Do yourself a favor and buy this product as soon as possible. I recommend it to everyone I know. It has saved me so much time!',
    'This product is overpriced and useless. It was a waste of money and it made all my hair fall out.',
    'Works like a dream and it is a bargain! It solves my problems with ease. I bought two!',
    'Do not buy! This product is a ripoff. I wish it was better, but it fails constantly. What a mistake!',
    'This thing is garbage. Do yourself a favor and save the money. Mine is a dumpster fire and fell apart.',
    'I adore this product. =) It makes my life so much easier. And it is a deal!'
]

In [9]:
# For each review in our `product_reviews` list
# Store a polarity score in `scores`
# Then print the score followed by the review
for review in product_reviews:
    scores = sid.polarity_scores(review)
    print(scores['compound'], review)

0.8979 I love this product. It helps me get so much work done. I tell everyone about what a great thing it is.
-0.2263 This product is defective. I feel like it is broken because it does not do what it promises. Do not buy this.
0.807 Do yourself a favor and buy this product as soon as possible. I recommend it to everyone I know. It has saved me so much time!
-0.6808 This product is overpriced and useless. It was a waste of money and it made all my hair fall out.
0.7772 Works like a dream and it is a bargain! It solves my problems with ease. I bought two!
-0.6792 Do not buy! This product is a ripoff. I wish it was better, but it fails constantly. What a mistake!
0.4404 This thing is garbage. Do yourself a favor and save the money. Mine is a dumpster fire and fell apart.
0.8799 I adore this product. =) It makes my life so much easier. And it is a deal!


In [12]:
text = "This thing is garbage Do yourself a favor and save the money Mine is a dumpster fire and fell apart"
for each in text.split():
    if each in sid.lexicon:
        print(each, sid.lexicon[each])
sid.polarity_scores(text)

favor 1.7
save 2.2
dumpster -0.6
fire -1.4


{'neg': 0.167, 'neu': 0.586, 'pos': 0.247, 'compound': 0.4404}

In [4]:
print(sid.polarity_scores("This is good"))
print(sid.polarity_scores("This is Good"))
print(sid.polarity_scores("This is GOOD"))

{'neg': 0.0, 'neu': 0.408, 'pos': 0.592, 'compound': 0.4404}
{'neg': 0.0, 'neu': 0.408, 'pos': 0.592, 'compound': 0.4404}
{'neg': 0.0, 'neu': 0.355, 'pos': 0.645, 'compound': 0.5622}


In [6]:
like = "I like this"
dont_like = "I do not like this"
print(sid.polarity_scores(like))
print(sid.polarity_scores(dont_like))

{'neg': 0.0, 'neu': 0.286, 'pos': 0.714, 'compound': 0.3612}
{'neg': 0.413, 'neu': 0.587, 'pos': 0.0, 'compound': -0.2755}


In [8]:
like = "I like this!"
dont_like = "I do not like this!"
print(sid.polarity_scores(like))
print(sid.polarity_scores(dont_like))

{'neg': 0.0, 'neu': 0.264, 'pos': 0.736, 'compound': 0.4199}
{'neg': 0.445, 'neu': 0.555, 'pos': 0.0, 'compound': -0.3404}


In [9]:
like = "I LIKE this"
dont_like = "I do not LIKE this"
print(sid.polarity_scores(like))
print(sid.polarity_scores(dont_like))

{'neg': 0.0, 'neu': 0.236, 'pos': 0.764, 'compound': 0.4995}
{'neg': 0.469, 'neu': 0.531, 'pos': 0.0, 'compound': -0.3924}


In [10]:
like = "I LIKE this"
dont_like = "I do NOT like this"
print(sid.polarity_scores(like))
print(sid.polarity_scores(dont_like))

{'neg': 0.0, 'neu': 0.236, 'pos': 0.764, 'compound': 0.4995}
{'neg': 0.413, 'neu': 0.587, 'pos': 0.0, 'compound': -0.2755}


In [11]:
like = "I like this"
dont_like = "I do NOT LIKE this"
print(sid.polarity_scores(like))
print(sid.polarity_scores(dont_like))

{'neg': 0.0, 'neu': 0.286, 'pos': 0.714, 'compound': 0.3612}
{'neg': 0.469, 'neu': 0.531, 'pos': 0.0, 'compound': -0.3924}


In [19]:
notbad = "not bad"
sid.polarity_scores(notbad)

{'neg': 0.0, 'neu': 0.26, 'pos': 0.74, 'compound': 0.431}

<h2>Load data from CSV file </h2>

In [3]:
df = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Manager_EN/data.csv", sep=";")

In [4]:
df.head()

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
0,703142204341026818,349287204,2016-02-26 08:58:59+00:00,703142204341026818,en,0,0,1,0,Twitter Web Client,[None],[None],1/2 I see Wirecard is rebounding after CEO Mar...
1,695258744737824768,282981965,2016-02-04 14:52:56+00:00,695258744737824768,en,1,0,0,0,Twitter for iPhone,['retweeted'],['695237701386244098'],RT @UrgentSpeed: Disrupt the Banks! (Jeff Stew...
2,695255175334924288,428593285,2016-02-04 14:38:45+00:00,695255175334924288,en,0,0,0,0,IFTTT,[None],[None],Thank you for following us Markus Braun!
3,695237701386244098,95129352,2016-02-04 13:29:19+00:00,695237701386244098,en,1,0,1,0,Twitter Web Client,[None],[None],"Disrupt the Banks! (Jeff Stewart, Markus Braun..."
4,719940834028863489,3040068274,2016-04-12 17:30:45+00:00,719940834028863489,en,0,0,0,0,metal archives updates,[None],[None],Artist updated: Markus Braun https://t.co/udZM...


In [5]:
len(df)

17801

In [28]:
for each in range(15):
    print(sid.polarity_scores(df.iloc[each]["text"]))
    print(df.iloc[each]["text"])
    print(each)

{'neg': 0.082, 'neu': 0.818, 'pos': 0.1, 'compound': 0.0992}
1/2 I see Wirecard is rebounding after CEO Markus Braun bought a chunk of shares. This is not a surprise, standard anti-short behaviour.
0
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
RT @UrgentSpeed: Disrupt the Banks! (Jeff Stewart, Markus Braun, Christian Angermayer, Harry Nelis) | #DLD16 https://t.co/WkFm9puWy0 cc @le…
1
{'neg': 0.0, 'neu': 0.682, 'pos': 0.318, 'compound': 0.4199}
Thank you for following us Markus Braun!
2
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Disrupt the Banks! (Jeff Stewart, Markus Braun, Christian Angermayer, Harry Nelis) | #DLD16 https://t.co/WkFm9puWy0 cc @lenddofriend
3
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Artist updated: Markus Braun https://t.co/udZMHaDsWi
4
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
$WDI #wirecard CEO Markus Braun on Q4 2015 Results - Yesterday's Earnings Call Transcript: https://t.co/E2fmw0rtPS
5
{'neg': 0.0, 'neu': 1.0, 'pos'

In [11]:
text = df.iloc[168]["text"]

In [13]:
print(text)

@_MarkusBraun HAVE A BLESSED DAY. HOPE YOU WOULD HAVE A READ ON MY BLOG. THANK YOU VERY MUCH. 😊😊😊https://t.co/SNufODP63F


In [20]:
text_1 = "@_MarkusBraun HAVE A BLESSED DAY. HOPE YOU WOULD HAVE A READ ON MY BLOG. THANK YOU VERY MUCH. 😊😊😊"
text_2 = "@_MarkusBraun HAVE A BLESSED DAY. HOPE YOU WOULD HAVE A READ ON MY BLOG. THANK YOU VERY MUCH."
text_3 = "@_MarkusBraun HAVE A BLESSED DAY. HOPE YOU WOULD HAVE A READ ON MY BLOG. THANK YOU VERY MUCH. :)"
print(text_1)
print(text_2)
print(text_3)

@_MarkusBraun HAVE A BLESSED DAY. HOPE YOU WOULD HAVE A READ ON MY BLOG. THANK YOU VERY MUCH. 😊😊😊
@_MarkusBraun HAVE A BLESSED DAY. HOPE YOU WOULD HAVE A READ ON MY BLOG. THANK YOU VERY MUCH.
@_MarkusBraun HAVE A BLESSED DAY. HOPE YOU WOULD HAVE A READ ON MY BLOG. THANK YOU VERY MUCH. :)


In [21]:
print(sid.polarity_scores(text))
print(sid.polarity_scores(text_1))
print(sid.polarity_scores(text_2))
print(sid.polarity_scores(text_3))

{'neg': 0.0, 'neu': 0.549, 'pos': 0.451, 'compound': 0.91}
{'neg': 0.0, 'neu': 0.549, 'pos': 0.451, 'compound': 0.91}
{'neg': 0.0, 'neu': 0.531, 'pos': 0.469, 'compound': 0.91}
{'neg': 0.0, 'neu': 0.457, 'pos': 0.543, 'compound': 0.9475}


In [32]:
text_4 = "Looking forward to one of our #spotlight speakers Markus Braun, CEO of @wirecard: Drop by!"
print(text_4)
sid.polarity_scores(text_4)

Looking forward to one of our #spotlight speakers Markus Braun, CEO of @wirecard: Drop by!


{'neg': 0.146, 'neu': 0.854, 'pos': 0.0, 'compound': -0.3382}

In [6]:
#re.findall(r"[\w']+", DATA)
#print(re.findall(r"[\w']+", df.loc[4]["tweet_type"])[0])
print(re.findall(r"\w+", df.loc[4]["tweet_type"])) 

['retweeted']


In [7]:
print(re.findall(r"\w+", df.at[4, "tweet_type"]))

['retweeted']


In [8]:
tweet_type_list = re.findall(r"\w+", df.at[4, "tweet_type"])

In [9]:
df.at[4,"tweet_type"] = tweet_type_list

In [11]:
df.at[4,"tweet_type"][0]

'retweeted'

In [24]:
for each in range(len(df)):
    df.at[each,"tweet_type"] = re.findall(r"\w+", df.at[each, "tweet_type"])
    #print(each)

In [37]:
tweet_type_list = ["None"]
type(tweet_type_list)

list

In [44]:
len(df["tweet_type"][1]) > 1

False

In [52]:
counter = 0
multiple_tweet_types = []
for each in range(len(df)):
    if len(df["tweet_type"][each]) > 1:
        multiple_tweet_types.append(each)
        counter += 1
counter

143

In [55]:
df.iloc[multiple_tweet_types]

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
723,1103965745619308544,1108781456325820423,2019-03-22 08:26:03+00:00,1109008324480110593,de,0,0,0,0,Twitter Web App,"[quoted, replied_to]","['1108985591214075904', '1106601531355340801']",@_MarkusBraun @oderman https://t.co/sFgN9ytJKs...
1186,1124324194521624576,172082646,2019-05-04 11:18:48+00:00,1124634476896825344,de,0,1,0,0,Twitter Web Client,"[quoted, replied_to]","['1124629815095590912', '1124629815095590912']",@Tradefair6 @boker_andre @WirecardCareers @wir...
1435,1151508892637114373,3011988771,2019-10-22 19:27:02+00:00,1186725670589779968,de,0,0,0,0,Twitter for Android,"[quoted, replied_to]","['1186476374242009089', '1151508892637114373']",@_MarkusBraun Europa eine neue Diktaturhttps:/...
1553,1197164173253324800,953644815568244736,2019-11-20 17:36:53+00:00,1197207197593350145,de,0,0,0,0,Twitter for iPhone,"[quoted, replied_to]","['1196607777935773696', '1197164173253324800']",@_MarkusBraun Alipay arbeitet mit einem Kranke...
3084,1266242840432013313,1077879160096374785,2020-05-29 07:04:16+00:00,1266264075492708353,de,0,0,3,0,Twitter for Android,"[quoted, replied_to]","['1266049241664937984', '1266242840432013313']",@MaxSteppan @_MarkusBraun Die Diskussion hatte...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9729,1286574929530560512,1118219555967520769,2020-07-24 08:12:46+00:00,1286575036246249473,de,0,0,0,0,Twitter for iPhone,"[quoted, replied_to]","['1101373999244812289', '1286574929530560512']",@CFOMG @FinancialTimes Der Ex-Wirecard-Chef @_...
9745,1286540503513792513,872914590300090368,2020-07-24 07:25:27+00:00,1286563128679989249,de,1,3,16,0,Twitter for iPhone,"[quoted, replied_to]","['1101373999244812289', '1286540503513792513']",@gaborsteingart Heute noch in Erinnerung: das ...
9872,1286314562371948544,774193248671858689,2020-07-23 15:01:12+00:00,1286315431872036865,de,0,1,0,0,Twitter Web App,"[quoted, replied_to]","['1101373999244812289', '1286314562371948544']",@HugoMuellerVogg @gaborsteingart Markus Braun ...
13285,1281849806223278080,2752407035,2020-07-11 08:48:07+00:00,1281872889273253888,de,1,1,6,0,Twitter for iPad,"[quoted, replied_to]","['1281705000125845504', '1281849806223278080']",... daß dem #IbizaUA laut JM @Alma_Zadic nur V...
